In [199]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# scraping data from wikipedia

In [200]:
df=pandas.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [201]:
df=df[0]

In [202]:
df.columns=df.loc[0,:]

In [203]:
df=df.drop(index=0).reindex()

In [204]:
df=df[df['Borough']!='Not assigned'].reset_index(drop=True)

In [205]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [206]:
df.shape

(103, 3)

In [207]:
# load geo info

In [208]:
# get LAT LNG INFO

In [209]:
import geocoder
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [210]:
# def get_lat_lng(Postal):
#     #from geopy.geocoders import Nominatim
#     geolocator = Nominatim(user_agent="my_user_agent1")
#     geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#     city ="Toronto"
#     country ="Canada"
#     loc=None
#     i=0
#     while (loc==None and i<5):
#         i+=1
#         #g=geocoder.google('{}, Toronto, Canada'.format(Postal))
#         #postal='M4A'
#         loc = geocode(Postal +','+ city+','+ country)
#     if loc!=None:
#         print(loc.latitude,loc.longitude)
#         return loc.latitude,loc.longitude
#     else :
#         return None,None

In [211]:
# LAT=[]
# LNG=[]
# for i in df.index:
#     print (i,df.loc[i,'Postal Code'])
#     lt,lg=get_lat_lng(df.loc[i,'Postal Code']+','+df.loc[i,'Neighbourhood'])
#     LAT.append(lt)
#     LNG.append(lg)
# df['Lat']=pd.Series(LAT)
# df['Lng']=pd.Series(Lng)


# actually not worked so i use csv file

In [212]:
df1=pd.read_csv('Geospatial_Coordinates.csv')

In [213]:
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [214]:
df2=df.join(df1.set_index('Postal Code'),'Postal Code',how='left')

In [215]:
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [216]:
import folium

In [217]:
folium_map= folium.Map(location=[43.753259,-79.329656],zoom_start=11)

In [218]:
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(folium_map)  

In [219]:
folium_map

# Define credintal infos for foursquare

In [226]:
CLIENT_ID = 'GJMSGCI4CGKD2Q22Y1HZ1E5PLFGYAUJTDYCE2KMENTROJ52L' # your Foursquare ID
CLIENT_SECRET = '2RD5KLXJGNU4QGPF243UI4URNTBG5BJJSQMEQDNA32VAGRAM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GJMSGCI4CGKD2Q22Y1HZ1E5PLFGYAUJTDYCE2KMENTROJ52L
CLIENT_SECRET:2RD5KLXJGNU4QGPF243UI4URNTBG5BJJSQMEQDNA32VAGRAM


In [227]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# function to get venues in neighborhood

In [228]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    #print(names, latitudes, longitudes)
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #print(url)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    #print([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [229]:
Toronto_venues = getNearbyVenues(names=df2['Neighbourhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

# group the neighborhood accourding to venues

In [230]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [231]:
Toronto_grouped=Toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [232]:
Toronto_grouped.iloc[1,1:].sort_values(ascending=False).index[:10].values

array(['Pizza Place', 'Pharmacy', 'Gym', 'Sandwich Place', 'Coffee Shop',
       'Pub', 'Distribution Center', 'Dessert Shop', 'Dim Sum Restaurant',
       'Diner'], dtype=object)

In [233]:
venue_max_num=10

Toronto_common_venues=pd.DataFrame(columns=['Neighborhood','1st','2nd','3rd']+[str(i) +'th' for i in range(4,11)])
Toronto_common_venues['Neighborhood']=Toronto_grouped['Neighborhood']

In [234]:
for i in Toronto_grouped.index:
    Toronto_common_venues.iloc[i,1:]=Toronto_grouped.iloc[i,1:].sort_values(ascending=False).index[:venue_max_num].values

# final data frames to be used

In [235]:
Toronto_common_venues.head()

,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Clothing Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Sandwich Place,Coffee Shop,Pub,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Frozen Yogurt Shop,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Restaurant,Deli / Bodega,Supermarket
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Women's Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Grocery Store,Thai Restaurant,Comfort Food Restaurant,Juice Bar,Butcher,Restaurant,Café


In [236]:
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [237]:
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# clustering the neighborhood

In [ ]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:95] 

In [ ]:
Toronto_common_venues['cluster']=kmeans.labels_

In [ ]:
Toronto_common_venues.head()

In [ ]:
Toronto_common_venues=Toronto_common_venues.join(df2.set_index('Neighbourhood'),on='Neighborhood')

In [ ]:

# create map
map_clusters = folium.Map(location=[43.753259,-79.329656], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_common_venues['Latitude'], Toronto_common_venues['Longitude'], Toronto_common_venues['Neighborhood'], Toronto_common_venues['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters